In [1]:
import csv
import json
import geopandas as gp
import pandas as pd
from datetime import datetime
from collections import Counter

In [2]:
#reads the TriCOG Land Bank service area footprint

tricog = gp.read_file("../data/tricog_dissolved.zip")

In [3]:
#read the allegheny county parcel file

allegheny_county_parcels = gp.read_file("../data/alleghenycounty_parcels202504.geojson")

In [4]:
#perform a spatial join between the TriCOG footprint and the county's parcels to find parcels within TriCOG

parcels_within_tricog = gp.sjoin(allegheny_county_parcels, tricog.to_crs("EPSG:4326"))

In [5]:
clipped_parcels = gp.clip(allegheny_county_parcels, tricog.to_crs("EPSG:4326"))

In [6]:
#read the assessment file and rename the parcel ID column "PIN"
with open("../data/assessments_apr_2025.csv", newline="") as f:
    allegheny_county_assessments_list = [row for row in csv.reader(f, delimiter=',')]
allegheny_county_assessments_list[0][0] = 'PIN'


In [7]:
#convert the list of lists to a dataframe
allegheny_county_assessments = pd.DataFrame(allegheny_county_assessments_list[1:], columns = allegheny_county_assessments_list[0])

In [8]:
assessments_on_clipped = clipped_parcels.merge(allegheny_county_assessments, how="inner", on="PIN")

In [9]:
#convert sale data dates to datetime and only keep houses that were last sold before 2022
#since we need houses with at least three years' tax delinquency, houses sold after 2022 could not meet that criteria

assessments_on_clipped['SALEDATE'] = pd.to_datetime(assessments_on_clipped['SALEDATE'], format='%m-%d-%Y')
assessments_on_clipped['PREVSALEDATE'] = pd.to_datetime(assessments_on_clipped['PREVSALEDATE'], format='%m-%d-%Y')
cutoff = datetime(2022,1,1)
older_sales = assessments_on_clipped[assessments_on_clipped['SALEDATE']<cutoff]

In [10]:
#creates a dict to use as a lookup to find the sale date of a parcel given its parcel id

pin_to_sale_date = {row.PIN: row.SALEDATE for row in older_sales.itertuples()}

In [11]:
#read the liens file
with open("../data/liens.csv", newline='') as f:
    liens_list = [row for row in csv.reader(f, delimiter=',')]

#convert the liens file to a dataframe
liens = pd.DataFrame(liens_list[1:], columns = liens_list[0])

In [12]:
#look for liens associated with any parcel that was sold before 2022 and
#is in the TriCOG footprint

liens_on_older_sales = liens[liens.pin.isin(older_sales.PIN)]

In [13]:
#creates a pandas DataFrame of liens from before 2022 that are associated with the current owner (and not liens that are attached to previous owners)

liens_on_current_owners = []
for row in liens_on_older_sales.itertuples():
    if int(row.tax_year) > pin_to_sale_date[row.pin].year:
        liens_on_current_owners.append(row)
liens_on_current_owners = pd.DataFrame(liens_on_current_owners)

In [14]:
#creates a dataframe slice of assessments of relevant parcels that have never satisfied a lien

satisfied_liens = liens_on_current_owners[liens_on_current_owners['satisfied']=='t']
outstanding_liens = liens_on_current_owners[~liens_on_current_owners.pin.isin(satisfied_liens.pin)]
assessments_with_outstanding_liens = older_sales[older_sales.PIN.isin(outstanding_liens.pin)]

In [15]:
#isolates parcels that are either vacant lots or single-family houses

residential_parcels = assessments_with_outstanding_liens[assessments_with_outstanding_liens.CLASSDESC=='RESIDENTIAL']
vacant_eligible_parcels = residential_parcels[residential_parcels['USEDESC'].isin(['RESIDENTIAL VACANT LAND', 'VACANT LAND'])]
single_family_parcels = residential_parcels[residential_parcels.USEDESC.isin(['SINGLE FAMILY'])]

In [16]:
#reads the MVA file, isolates DEF MVAs within TriCOG footprint

mva = gp.read_file("../data/pitts_allegheny_mva2021.zip")
def_mvas = mva[mva.MVA21.isin(['D','E','F'])]
mvas_in_tcog = gp.clip(def_mvas, tricog.to_crs("EPSG:4326"))

In [17]:
#isolates relevant single-family housing parcels within DEF MVAs in TriCOG boundaries

single_family_in_mva = gp.clip(single_family_parcels, def_mvas)

In [18]:
blocks = gp.read_file("../data/pa_blocks_2020.zip")

In [19]:
blocks_within_tcog = gp.clip(blocks.to_crs("EPSG:4326"), tricog.to_crs("EPSG:4326"))

In [20]:
blocks_with_single_family = gp.sjoin(blocks_within_tcog, single_family_in_mva, predicate="contains")
blocks_with_single_family = blocks_with_single_family.loc[:, blocks_within_tcog.columns].drop_duplicates()

In [43]:
all_parcels_in_single_family_blocks = gp.sjoin(blocks_with_single_family, assessments_on_clipped, predicate="contains")

In [109]:
c = Counter(all_parcels_in_single_family_blocks.GEOID)
blocks_to_parcel_count = {k: {'parcel_count': v, 'eligible_count': 0, 'vacant_count': 0, 'vacant_percentage': 0} for k, v in c.items()}

for row in all_parcels_in_single_family_blocks.itertuples():
    if row.PIN in list(vacant_eligible_parcels.PIN):
        blocks_to_parcel_count[row.GEOID]['vacant_count'] += 1
    if row.PIN in list(single_family_in_mva.PIN):
        blocks_to_parcel_count[row.GEOID]['eligible_count'] += 1

for block in blocks_to_parcel_count:
    block_dict = blocks_to_parcel_count[block]
    block_dict['vacant_percentage'] = block_dict['vacant_count']/block_dict['parcel_count']

In [ ]:
#NOTE: This cell will likely be deleted, but I am not ready to do something that rash yet.


#perform an inner join that connects the parcels within TriCOG's footprint and their assessment data

# assessments_within_tricog = parcels_within_tricog.merge(allegheny_county_assessments, how="inner", on="PIN")

#convert sale data dates to datetime and only keep houses that were last sold before 2022
#since we need houses with at least three years' tax delinquency, houses sold after 2022 could not meet that criteria

# assessments_within_tricog['SALEDATE'] = pd.to_datetime(assessments_within_tricog['SALEDATE'], format='%m-%d-%Y')
# assessments_within_tricog['PREVSALEDATE'] = pd.to_datetime(assessments_within_tricog['PREVSALEDATE'], format='%m-%d-%Y')
# cutoff = datetime(2022,1,1)
# older_sales = assessments_within_tricog[assessments_within_tricog['SALEDATE']<cutoff]